In [1]:
import numpy as np

In this notebook, we show how to use the two emulators to predict the Figure-of-merit of the 3x2pt probes as a function of 6 different features (area, depth, $\sigma(m)$, $\sigma_z$, $\sigma(\Delta_z)$, $\sigma(\sigma_z)$) for a given LSST survey scenario.

First, load the data from the `fom_data.npy` file included in this repository. 

In [13]:
fom_data = np.load('fom_data.npy')

fom     = fom_data[:,1]
features = fom_data[:,2:]

# area    = x[:,2]
# i_depth = x[:,3]
# sigma_m = x[:,4]
# sigma_z = x[:,5]
# sigma_delta_z = x[:,6]
# sigma_sigma_z = x[:,7]

We can use our Polynomial Chaos Expansion (PCE) emulator in the following way to predict the Figure-of-merit

In [ ]:
# Use the first 33 points to train the emulator and then predict the F-o-M for the remaining three points

train_features = features[:33]
train_fom      = fom[:33]

test_features = features[33:]
test_fom      = fom[33:]

# The emulators are usually easier to train by normalizing the features to within (0, 1)

def do_feature_normalization(features):
    return (features - prior_lims[:,0][np.newaxis]) / prior_range[np.newaxis]

train_features_norm = do_feature_normalization(train_features)

In [ ]:
# We need a limit of the each of the features for our PCE emulator

prior_lims  = np.array([features.min(0), features.max(0)]).T
prior_range = prior_lims[:,1] - prior_lims[:,0]

In [4]:
# Our feature vector is 6-dimensional (area, depth, ...)
N_DIM = 6

from emulator import PCEEmulator
pce_emu = PCEEmulator(N_DIM, prior_lims)

In [16]:
pce_emu.train(train_features_norm, train_fom)

In [17]:
def print_feature(test_feature):
    labels = ['Area     ', 'Depth     ', 'sigma(m)', 'sigma_z    ', 'sigma(Delta_z)', 'sigma_sigma_z']
    for label, feature in zip(labels, test_feature):
        print(label + '\t: %2.4f'%(feature))

In [9]:
for test_feature, fom_true in zip(test_features, test_fom):
    print_feature(test_feature)
    test_feature_norm = do_feature_normalization(test_feature)
    fom_pred = pce_emu.predict(test_feature_norm)
    print("===========================")
    print("Fom (true): %2.3f, Fom (pred): %2.3f, "%(fom_true, fom_pred))
    print("===========================")

Area     	: 14060.0000
Depth     	: 26.1100
sigma(m)	: 0.0035
sigma_z    	: 0.0690
sigma(Delta_z)	: 0.0039
sigma_sigma_z	: 0.0036
Fom (true): 24.120, Fom (pred): 28.424, 
Area     	: 10919.0000
Depth     	: 25.9500
sigma(m)	: 0.0077
sigma_z    	: 0.0560
sigma(Delta_z)	: 0.0015
sigma_sigma_z	: 0.0059
Fom (true): 24.648, Fom (pred): 25.970, 
Area     	: 13213.0000
Depth     	: 26.1900
sigma(m)	: 0.0135
sigma_z    	: 0.0150
sigma(Delta_z)	: 0.0036
sigma_sigma_z	: 0.0042
Fom (true): 29.649, Fom (pred): 32.684, 


Alternatively, we can use the Gaussian process emulator for the same. Usually the Gaussian process emulator is more robust, which the PCE emulator may require tuning of a few parameters.

### GP emulator

In [10]:
from emulator import GPEmulator
gp_emu = GPEmulator(N_DIM)

In [11]:
gp_emu.train(train_features_norm, train_fom)

Optimization terminated successfully.
         Current function value: 18.671423
         Iterations: 9
         Function evaluations: 700


In [12]:
for test_feature, fom_true in zip(test_features, test_fom):
    print("Test feature: ")
    print("------------")
    print_feature(test_feature)
    test_feature_norm = do_feature_normalization(test_feature)
    fom_pred = gp_emu.predict(test_feature_norm)
    print("===========================")
    print("Fom (true): %2.3f, Fom (pred): %2.3f, "%(fom_true, fom_pred))
    print("===========================")

Test feature: 
------------
Area     	: 14060.0000
Depth     	: 26.1100
sigma(m)	: 0.0035
sigma_z    	: 0.0690
sigma(Delta_z)	: 0.0039
sigma_sigma_z	: 0.0036
Fom (true): 24.120, Fom (pred): 26.135, 
Test feature: 
------------
Area     	: 10919.0000
Depth     	: 25.9500
sigma(m)	: 0.0077
sigma_z    	: 0.0560
sigma(Delta_z)	: 0.0015
sigma_sigma_z	: 0.0059
Fom (true): 24.648, Fom (pred): 27.819, 
Test feature: 
------------
Area     	: 13213.0000
Depth     	: 26.1900
sigma(m)	: 0.0135
sigma_z    	: 0.0150
sigma(Delta_z)	: 0.0036
sigma_sigma_z	: 0.0042
Fom (true): 29.649, Fom (pred): 29.128, 
